In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import joblib


In [3]:
df = pd.read_csv('../dataset/tagged.csv',  delimiter=',', quoting=2, encoding="utf-8")

In [4]:
df.head(5)

,id,tweets,Polarity,Subjectivity
0,0.0,ChatGPT: Optimizing Language Models for Dialog...,0.000000,0.000000
1,1.0,"Try talking with ChatGPT, our new AI system wh...",0.136364,0.454545
2,2.0,ChatGPT: Optimizing Language Models for Dialog...,0.000000,0.000000
3,3.0,"THRILLED to share that ChatGPT, our new model ...",0.302273,0.479242
4,4.0,"As of 2 minutes ago, @OpenAI released their ne...",0.211039,0.495130


In [5]:
def procces_sentence(review:str)->str:
    sample_review = re.sub(r'http\S+', '', review)
    sample_review = re.sub("[^a-zA-Z]",' ',sample_review)
    sample_review = sample_review.lower()
    sample_review = sample_review.split()
    swords = set(stopwords.words("english"))                     
    sample_review = [w for w in sample_review if w not in swords and len(w) > 1]
    sample_review = " ".join(sample_review)
    return sample_review

def take_res(count:int)->list:
    sub = df["Subjectivity"][count]
    pol = df["Polarity"][count]
    return [pol, sub]
    
    


In [6]:
from tqdm import tqdm

count_t = 219288

trainx = []
trainy = []
for i in tqdm(range(219200)):
    trainx.append(procces_sentence(df["tweets"][i]))
    trainy.append(take_res(i))



100%|██████████| 219200/219200 [00:36<00:00, 6067.88it/s]


In [7]:
trainy = np.array(trainy)
train_x, test_x, y_train, y_test = train_test_split(trainx,trainy, test_size = 0.1)
vectorizer = CountVectorizer( max_features = 40000 )
train_x = vectorizer.fit_transform(train_x)



In [8]:
test_xx = vectorizer.transform(test_x)
test_xx
test_xx = test_xx.toarray()

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np

models = {

    'Linear Regressor': MultiOutputRegressor(LinearRegression()),
    # 'Random Forest Regressor': MultiOutputRegressor(RandomForestRegressor()),
    # 'Extra Trees Regressor': MultiOutputRegressor(ExtraTreesRegressor()),
    'AdaBoost Regressor': MultiOutputRegressor(AdaBoostRegressor()),
    # 'Decision Tree Regressor': MultiOutputRegressor(DecisionTreeRegressor()),
    'K Neighbors Regressor': MultiOutputRegressor(KNeighborsRegressor()),
    'CatBoost Regressor': MultiOutputRegressor(CatBoostRegressor(learning_rate=0.1, depth=6, iterations=500, verbose=0)),
    'xgboost': MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror')),    

}

results_regression = {}

for name, model in models.items():
    model.fit(train_x, y_train)
    y_pred = model.predict(test_xx)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results_regression[name] = {'RMSE': rmse}
    joblib.dump(model, name)
    print(f'{name}: RMSE = {rmse}')



results_regression_df = pd.DataFrame(results_regression).T

print("Regression Results:")
print(results_regression_df)



Linear Regressor: RMSE = 0.23039232806310442/model saved
AdaBoost Regressor: RMSE = 0.2988164968719073/model saved
K Neighbors Regressor: RMSE = 0.28313674332722266/model saved
CatBoost Regressor: RMSE = 0.21024351889150003/model saved
xgboost: RMSE = 0.3797227306020127/model saved
Regression Results:
                           RMSE
Linear Regressor       0.230392
AdaBoost Regressor     0.298816
K Neighbors Regressor  0.283137
CatBoost Regressor     0.210244
xgboost                0.379723


Sentiment probabilities for the input: -0.5178470748209099
